In [10]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

In [11]:
class Disciplina:
    def __init__(self, codigo, nome, creditos):
        self.codigo = codigo
        self.nome = nome
        self.creditos = creditos

In [12]:
class Curso:
    def __init__(self, nome, unidade, turno="N/A", modalidade="N/A", grau="N/A"):
        self.nome = nome
        self.unidade = unidade
        self.turno = turno
        self.modalidade = modalidade
        self.grau = grau
        self.disciplinas_obrigatorias = []
        self.disciplinas_optativas = []

In [13]:
class Unidade:
    def __init__(self, nome):
        self.nome = nome
        self.cursos = []
    def adicionar_curso(self, curso):
        self.cursos.append(curso)

In [14]:
# Funções auxiliares (exemplos)
def wait_for_select_options(driver, select_id):
    WebDriverWait(driver, 20).until(
        lambda d: len(d.find_element(By.ID, select_id).find_elements(By.TAG_NAME, 'option')) > 1
    )

def esperar_botao_buscar_habilitado(driver):
    return WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.ID, "btnBuscar"))
    )

def fechar_popup_erro_se_existir(driver):
    try:
        popup = driver.find_element(By.ID, "popupErro")
        if popup.is_displayed():
            fechar = popup.find_element(By.CLASS_NAME, "fechar")
            fechar.click()
            return True
    except:
        return False
    return False

def esperar_overlay_sumir(driver):
    WebDriverWait(driver, 30).until_not(
        EC.presence_of_element_located((By.CLASS_NAME, "overlay"))
    )

def esperar_overlay_e_clique(driver, elemento):
    esperar_overlay_sumir(driver)
    elemento.click()

def wait_for_grade_carregar(driver):
    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.ID, "gradeContainer"))
    )
    def esperar_e_clicar(driver, by, identificador, timeout=30):
    elemento = WebDriverWait(driver, timeout).until(
        EC.element_to_be_clickable((by, identificador))
    )
    elemento.click()

def extrair_disciplinas(soup):
    # Exemplo simplificado para extrair disciplinas da tabela
    disciplinas = []
    tabela = soup.find('table', {'id': 'tblDisciplinas'})
    if tabela:
        for tr in tabela.find_all('tr')[1:]:  # pular cabeçalho
            cols = tr.find_all('td')
            if len(cols) >= 3:
                codigo = cols[0].text.strip()
                nome = cols[1].text.strip()
                creditos = cols[2].text.strip()
                disciplinas.append(Disciplina(codigo, nome, creditos))
    return disciplinas


In [15]:

# Classe principal do scraper
class JupiterWebScraper:
    def __init__(self):
        self.unidades = []

    def carregar_dados(self, driver, num_unidades=None):
        URL = "https://uspdigital.usp.br/jupiterweb/jupCarreira.jsp?codmnu=8275"
        driver.get(URL)

        wait_for_select_options(driver, "comboUnidade")
        select_unidades = Select(driver.find_element(By.ID, "comboUnidade"))

        total_unidades = len(select_unidades.options) - 1  # ignorar primeira vazia
        limite = num_unidades if num_unidades else total_unidades

        for i in range(1, limite + 1):
            select_unidades.select_by_index(i)
            unidade_nome = select_unidades.first_selected_option.text.strip()
            print(f"\nProcessando unidade: {unidade_nome}")
            unidade = Unidade(unidade_nome)

            wait_for_select_options(driver, "comboCurso")
            select_cursos = Select(driver.find_element(By.ID, "comboCurso"))
            cursos_options = [opt for opt in select_cursos.options if opt.get_attribute('value') != '']

            for option in cursos_options:
                try:
                    select_cursos.select_by_value(option.get_attribute('value'))
                    curso_nome = option.text.strip()

                    buscar_btn = esperar_botao_buscar_habilitado(driver)
                    buscar_btn.click()

                    WebDriverWait(driver, 30).until(
                        EC.element_to_be_clickable((By.ID, "step4-tab"))
                    )

                    if fechar_popup_erro_se_existir(driver):
                        continue

                    esperar_overlay_sumir(driver)
                    grade_tab = driver.find_element(By.ID, "step4-tab")
                    esperar_overlay_e_clique(driver, grade_tab)

                    wait_for_grade_carregar(driver)
                    soup = BeautifulSoup(driver.page_source, 'html.parser')
                    disciplinas = extrair_disciplinas(soup)

                    curso = Curso(curso_nome, unidade_nome)
                    curso.disciplinas_obrigatorias.extend(disciplinas)

                    unidade.adicionar_curso(curso)

                    aba_dados = driver.find_element(By.ID, "step1-tab")
                    esperar_overlay_e_clique(driver, aba_dados)

                except Exception as e:
                    print(f"Erro no curso {option.text}: {e}")
                    aba_dados = driver.find_element(By.ID, "step1-tab")
                    esperar_overlay_e_clique(driver, aba_dados)

            self.unidades.append(unidade)

In [16]:
  # Métodos do menu:
def listar_cursos_por_unidade(self):
    for unidade in self.unidades:
        print(f"\nUnidade: {unidade.nome}")
        for curso in unidade.cursos:
            print(f" - {curso.nome}")

def dados_curso(self, nome_curso):
    for unidade in self.unidades:
        for curso in unidade.cursos:
            if nome_curso.lower() in curso.nome.lower():
                print(f"\nCurso: {curso.nome} (Unidade: {unidade.nome})")
                print("Disciplinas obrigatórias:")
                for d in curso.disciplinas_obrigatorias:
                    print(f"  {d.codigo} - {d.nome} ({d.creditos} créditos)")
                return
    print("Curso não encontrado.")

def dados_todos_cursos(self):
    for unidade in self.unidades:
        for curso in unidade.cursos:
            print(f"\nCurso: {curso.nome} (Unidade: {unidade.nome})")
            print("Disciplinas obrigatórias:")
            for d in curso.disciplinas_obrigatorias:
                print(f"  {d.codigo} - {d.nome} ({d.creditos} créditos)")

def dados_disciplina(self, nome_ou_codigo_disciplina):
    encontrados = []
    for unidade in self.unidades:
        for curso in unidade.cursos:
            for d in curso.disciplinas_obrigatorias:
                if nome_ou_codigo_disciplina.lower() in d.nome.lower() or nome_ou_codigo_disciplina.lower() in d.codigo.lower():
                    encontrados.append((d, curso, unidade))
    if not encontrados:
        print("Disciplina não encontrada.")
    else:
        for d, curso, unidade in encontrados:
            print(f"\nDisciplina: {d.codigo} - {d.nome} ({d.creditos} créditos)")
            print(f"Curso: {curso.nome} (Unidade: {unidade.nome})")

def disciplinas_multiuso(self):
    contador = {}
    for unidade in self.unidades:
        for curso in unidade.cursos:
            for d in curso.disciplinas_obrigatorias:
                contador[d.codigo] = contador.get(d.codigo, 0) + 1
    multiuso = {k:v for k,v in contador.items() if v > 1}
    if not multiuso:
        print("Nenhuma disciplina usada em mais de um curso.")
    else:
        print("Disciplinas usadas em mais de um curso:")
        for codigo, count in multiuso.items():
            print(f"{codigo}: {count} cursos")

def cursos_com_mais_disciplinas(self, top_n=5):
    cursos_com_count = []
    for unidade in self.unidades:
        for curso in unidade.cursos:
            cursos_com_count.append((curso.nome, len(curso.disciplinas_obrigatorias)))
    cursos_com_count.sort(key=lambda x: x[1], reverse=True)
    print(f"Top {top_n} cursos com mais disciplinas:")
    for nome, count in cursos_com_count[:top_n]:
        print(f"{nome}: {count} disciplinas")

def unidades_com_mais_cursos(self, top_n=5):
    unidades_count = [(u.nome, len(u.cursos)) for u in self.unidades]
    unidades_count.sort(key=lambda x: x[1], reverse=True)
    print(f"Top {top_n} unidades com mais cursos:")
    for nome, count in unidades_count[:top_n]:
        print(f"{nome}: {count} cursos")

def disciplinas_com_mais_creditos(self, top_n=5):
    disciplinas_dict = {}
    for unidade in self.unidades:
        for curso in unidade.cursos:
            for d in curso.disciplinas_obrigatorias:
                if d.codigo not in disciplinas_dict or float(d.creditos) > float(disciplinas_dict[d.codigo].creditos):
                    disciplinas_dict[d.codigo] = d
    sorted_disc = sorted(disciplinas_dict.values(), key=lambda x: float(x.creditos), reverse=True)
    print(f"Top {top_n} disciplinas com mais créditos:")
    for d in sorted_disc[:top_n]:
        print(f"{d.codigo} - {d.nome}: {d.creditos} créditos")


In [17]:
# Execução

def main():
    print("=== Bem-vindo ao Sistema de Consulta JupiterWeb ===")
    
    print("Iniciando o WebDriver...")
    driver = webdriver.Chrome()
    
    try:
        scraper = JupiterWebScraper()
        
        limite_unidades = input("Digite o número máximo de unidades a carregar (ou pressione Enter para carregar todas): ")
        limite_unidades = int(limite_unidades) if limite_unidades.strip().isdigit() else None
        
        scraper.carregar_dados(driver, limite_unidades)
        
        while True:
            print("\n--- Menu Principal ---")
            print("1. Listar cursos por unidade")
            print("2. Dados de um curso específico")
            print("3. Dados de todos os cursos")
            print("4. Dados de uma disciplina específica")
            print("5. Disciplinas usadas em mais de um curso")
            print("6. Top cursos com mais disciplinas")
            print("7. Top unidades com mais cursos")
            print("8. Top disciplinas com mais créditos")
            print("9. Sair")
            
            opcao = input("Escolha uma opção: ")
            
            if opcao == '1':
                scraper.listar_cursos_por_unidade()
            elif opcao == '2':
                nome_curso = input("Digite o nome do curso: ")
                scraper.dados_curso(nome_curso)
            elif opcao == '3':
                scraper.dados_todos_cursos()
            elif opcao == '4':
                nome_ou_codigo_disciplina = input("Digite o nome ou código da disciplina: ")
                scraper.dados_disciplina(nome_ou_codigo_disciplina)
            elif opcao == '5':
                scraper.disciplinas_multiuso()
            elif opcao == '6':
                top_n = input("Quantos cursos listar? (padrão: 5): ")
                top_n = int(top_n) if top_n.strip().isdigit() else 5
                scraper.cursos_com_mais_disciplinas(top_n)
            elif opcao == '7':
                top_n = input("Quantas unidades listar? (padrão: 5): ")
                top_n = int(top_n) if top_n.strip().isdigit() else 5
                scraper.unidades_com_mais_cursos(top_n)
            elif opcao == '8':
                top_n = input("Quantas disciplinas listar? (padrão: 5): ")
                top_n = int(top_n) if top_n.strip().isdigit() else 5
                scraper.disciplinas_com_mais_creditos(top_n)
            elif opcao == '9':
                print("Encerrando o programa. Até logo!")
                break
            else:
                print("Opção inválida. Tente novamente.")
                
    except Exception as e:
        print(f"Ocorreu um erro: {e}")
    finally:
        print("Encerrando o WebDriver...")
        driver.quit()

if __name__ == "__main__":
    main()

=== Bem-vindo ao Sistema de Consulta JupiterWeb ===
Iniciando o WebDriver...


Digite o número máximo de unidades a carregar (ou pressione Enter para carregar todas):  3



Processando unidade: Escola de Artes, Ciências e Humanidades - ( EACH )
Erro no curso Bacharelado em Biotecnologia (Ciclo Básico) - integral: Message: 
Stacktrace:
#0 0x573331e4eb9a <unknown>
#1 0x573331b34670 <unknown>
#2 0x573331b83c48 <unknown>
#3 0x573331b83ee1 <unknown>
#4 0x573331bca924 <unknown>
#5 0x573331ba8bad <unknown>
#6 0x573331bc7dd6 <unknown>
#7 0x573331ba8923 <unknown>
#8 0x573331b766e7 <unknown>
#9 0x573331b776de <unknown>
#10 0x573331e1866b <unknown>
#11 0x573331e1c611 <unknown>
#12 0x573331e044e5 <unknown>
#13 0x573331e1d192 <unknown>
#14 0x573331de96ef <unknown>
#15 0x573331e3d9d8 <unknown>
#16 0x573331e3dba7 <unknown>
#17 0x573331e4d9ec <unknown>
#18 0x723a7d09caa4 <unknown>
#19 0x723a7d129c3c <unknown>

Erro no curso Bacharelado em Educação Física e Saúde (Ciclo Básico) - integral: Message: 
Stacktrace:
#0 0x573331e4eb9a <unknown>
#1 0x573331b34670 <unknown>
#2 0x573331b83c48 <unknown>
#3 0x573331b83ee1 <unknown>
#4 0x573331bca924 <unknown>
#5 0x573331ba8bad <unk

KeyboardInterrupt: 